In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import bds
from scipy.stats import skew, kurtosis

daily_data_path = "/home/lianlian/models/Time-LLM/dataset/carbon/target_daily/1-historical_price_daily.xlsx"  # File with carbon credit price data
monthly_data_path = "/home/lianlian/models/Time-LLM/dataset/carbon/target_monthly/1-historical_price-Monthly.xlsx"  # File with carbon credit price data

In [2]:
# Load daily data
daily_data = pd.read_excel(daily_data_path)
print("Daily Data Head:")
daily_data.head()  # Display the first few rows of the daily data

Daily Data Head:


,Month-Year,Price,Volume
0,2012-11-13,8.53,1150000
1,2012-11-15,7.30,4031000
2,2012-11-16,6.75,3000000
3,2012-11-20,6.47,4478500
4,2012-11-20,6.56,868000


In [3]:
# Load monthly data
monthly_data = pd.read_excel(monthly_data_path)
print("Monthly Data Head:")
monthly_data.head()  # Display the first few rows of the monthly data

Monthly Data Head:


,Month-Year,Price,Volume
0,2012-01-01,6.973333,2350000
1,2012-02-01,8.516000,2200000
2,2012-03-01,7.498333,3050000
3,2012-04-01,6.718000,2200000
4,2012-05-01,6.547500,1200000


In [4]:
def describe_price(df, col='Price'):
    stats = {
        'Minimum': df[col].min(),
        'Maximum': df[col].max(),
        'Mean': df[col].mean(),
        'Standard Deviation': df[col].std(),
        'Skewness': skew(df[col], nan_policy='omit'),
        'Kurtosis': kurtosis(df[col], nan_policy='omit')
    }
    return stats

daily_stats = describe_price(daily_data)
monthly_stats = describe_price(monthly_data)

print("Daily:")
for key, value in daily_stats.items():
    print(f"{key}: {value}")
print()
print("Monthly:")
for key, value in monthly_stats.items():
    print(f"{key}: {value}")

Daily:
Minimum: 2.65
Maximum: 97.51
Mean: 31.76612916328188
Standard Deviation: 29.94002128196398
Skewness: 0.7545053596313961
Kurtosis: -1.004488603896276

Monthly:
Minimum: 3.418125
Maximum: 91.98055555555555
Mean: 22.309429847987005
Standard Deviation: 24.973447955725494
Skewness: 1.55322845516622
Kurtosis: 1.1296939802708632


## ADF tests

In [5]:
def comprehensive_adf_test(ts, max_lags=None, regression_type='ct'):
    """
    Conduct comprehensive ADF test with multiple specifications
    
    Parameters:
    ts (pd.Series): Time series data
    max_lags (int): Maximum number of lags to test (default: int(12*(len(ts)/100)^(1/4)))
    regression_type (str): 'c' (constant), 't' (trend), 'ct' (constant+trend), 'n' (none)
    
    Returns:
    dict: Test results
    """
    if max_lags is None:
        max_lags = int(12 * (len(ts) / 100) ** 0.25)
    
    # Test different regression specifications
    print(f"\n=== ADF Test with {regression_type} specification ===")
    
    # Automatic lag selection using AIC
    adf_result = adfuller(ts, maxlag=max_lags, regression=regression_type, autolag='AIC')
    
    results = {
        'adf_statistic': adf_result[0],
        'p_value': adf_result[1],
        'lags_used': adf_result[2],
        'observations': adf_result[3],
        'critical_values': adf_result[4],
        'ic_best': adf_result[5]
    }
    
    # print(f"ADF Statistic: {adf_result[0]:.6f}")
    # print(f"p-value: {adf_result[1]:.6f}")
    # print(f"Lags used: {adf_result[2]}")
    # print(f"Number of observations: {adf_result[3]}")
    # print(f"Critical Values:")
    # for key, value in adf_result[4].items():
    #     print(f"\t{key}: {value:.6f}")
    
    # # Interpretation
    # if adf_result[1] <= 0.05:
    #     print("Result: Reject null hypothesis - Series is stationary")
    # else:
    #     print("Result: Fail to reject null hypothesis - Series has unit root")
    
    return results

In [ ]:
def calculate_average_adf_across_windows(series, window_size=512):
    """
    Calculate average BDS statistic across all possible windows of given size
    
    Parameters:
    series (pd.Series): Time series data
    window_size (int): Size of sliding window
    dim (int): Dimension for BDS test
    
    Returns:
    float: Average BDS statistic across all windows
    """
    total_windows = len(series) - window_size + 1
    adf_stats = []
    pval_nonsig_count = 0
    
    for i in range(total_windows):
        window = series[i:i+window_size]
        adf_results = comprehensive_adf_test(window)
        adf_stats.append(adf_results['adf_statistic'])
        if adf_results['p_value'] < 0.05:
            pval_sig_count += 1
    
    return np.mean(adf_stats), pval_nonsig_count, total_windows

Daily data

In [ ]:
# Calculate average BDS statistics for different dimensions, daily
print("Average BDS Statistics across all windows of size 512 (daily):")
for dim in range(2, 6):
    avg_adf, pval_sig, n_windows = calculate_average_adf_across_windows(daily_data['Price'], window_size=512)
    print(f"Dimension {dim}: Avg. BDS Statistic = {avg_adf:.5f}, Number of nonsig. windows = {pval_sig}/{n_windows}")

In [6]:
# Test levels
print("\n" + "="*50)
print("TESTING LEVELS (PRICES)")
print("="*50)
adf_results_levels = comprehensive_adf_test(daily_data['Price'])


TESTING LEVELS (PRICES)

=== ADF Test with ct specification ===
ADF Statistic: -2.006029
p-value: 0.598146
Lags used: 24
Number of observations: 2437
Critical Values:
	1%: -3.962490
	5%: -3.412293
	10%: -3.128112
Result: Fail to reject null hypothesis - Series has unit root


In [7]:
# Test log levels
print("\n" + "="*50)
print("TESTING LOG LEVELS")
print("="*50)
adf_results_log = comprehensive_adf_test(np.log(daily_data['Price']))


TESTING LOG LEVELS

=== ADF Test with ct specification ===
ADF Statistic: -2.276293
p-value: 0.447105
Lags used: 15
Number of observations: 2446
Critical Values:
	1%: -3.962476
	5%: -3.412286
	10%: -3.128108
Result: Fail to reject null hypothesis - Series has unit root


Monthly data

In [8]:
# Test levels
print("\n" + "="*50)
print("TESTING LEVELS (PRICES)")
print("="*50)
adf_results_levels = comprehensive_adf_test(monthly_data['Price'])


TESTING LEVELS (PRICES)

=== ADF Test with ct specification ===
ADF Statistic: -1.643409
p-value: 0.774976
Lags used: 12
Number of observations: 121
Critical Values:
	1%: -4.035606
	5%: -3.447417
	10%: -3.148699
Result: Fail to reject null hypothesis - Series has unit root


In [9]:
# Test log levels
print("\n" + "="*50)
print("TESTING LOG LEVELS")
print("="*50)
adf_results_log = comprehensive_adf_test(np.log(monthly_data['Price']))


TESTING LOG LEVELS

=== ADF Test with ct specification ===
ADF Statistic: -2.435511
p-value: 0.360917
Lags used: 1
Number of observations: 132
Critical Values:
	1%: -4.029044
	5%: -3.444289
	10%: -3.146873
Result: Fail to reject null hypothesis - Series has unit root


## BDS tests

In [10]:
def bds_test_results(series, dim=2):
    bds_results = bds(series, max_dim=dim)
    return {
        'BDS statistic': bds_results[0].max() if dim == 2 else bds_results[0][dim-2],
        'p-value': bds_results[1].max() if dim == 2 else bds_results[1][dim-2],
    }

In [23]:
def calculate_average_bds_across_windows(series, window_size=512, dim=2):
    """
    Calculate average BDS statistic across all possible windows of given size
    
    Parameters:
    series (pd.Series): Time series data
    window_size (int): Size of sliding window
    dim (int): Dimension for BDS test
    
    Returns:
    float: Average BDS statistic across all windows
    """
    total_windows = len(series) - window_size + 1
    bds_stats = []
    pval_nonsig_count = 0
    
    for i in range(total_windows):
        window = series[i:i+window_size]
        bds_result = bds_test_results(window, dim=dim)
        bds_stats.append(bds_result['BDS statistic'])
        if bds_result['p-value'] >= 0.05:
            pval_nonsig_count += 1
    
    return np.mean(bds_stats), pval_nonsig_count, total_windows

In [30]:
print("BDS Test (Daily):")
for dim in range(2, 6):
    bds_daily = bds_test_results(daily_data['Price'][0:512], dim=dim)
    print(f"Dimension {dim}:", bds_daily)

BDS Test (Daily):
Dimension 2: {'BDS statistic': 97.23523543456281, 'p-value': 0.0}
Dimension 3: {'BDS statistic': 106.45474744132922, 'p-value': 0.0}
Dimension 4: {'BDS statistic': 117.66881994743484, 'p-value': 0.0}
Dimension 5: {'BDS statistic': 133.39370028121263, 'p-value': 0.0}


In [31]:
# Calculate average BDS statistics for different dimensions, daily
print("Average BDS Statistics across all windows of size 512 (daily):")
for dim in range(2, 6):
    avg_bds, pval_nonsig, n_windows = calculate_average_bds_across_windows(daily_data['Price'], window_size=512, dim=dim)
    print(f"Dimension {dim}: Avg. BDS Statistic = {avg_bds:.5f}, Number of nonsig. windows = {pval_nonsig}/{n_windows}")

Average BDS Statistics across all windows of size 512 (daily):
Dimension 2: Avg. BDS Statistic = 86.59026, Number of nonsig. windows = 0/1951
Dimension 3: Avg. BDS Statistic = 94.80863, Number of nonsig. windows = 0/1951
Dimension 4: Avg. BDS Statistic = 104.91681, Number of nonsig. windows = 0/1951
Dimension 5: Avg. BDS Statistic = 119.21401, Number of nonsig. windows = 0/1951


In [25]:
print("BDS Test (Monthly):")
for dim in range(2, 6):
    bds_monthly = bds_test_results(monthly_data['Price'], dim=dim)
    print(f"Dimension {dim}:", bds_monthly)

BDS Test (Monthly):
Dimension 2: {'BDS statistic': 20.44239745538141, 'p-value': 7.020437708002223e-93}
Dimension 3: {'BDS statistic': 21.289759630630535, 'p-value': 1.4125053373737697e-100}
Dimension 4: {'BDS statistic': 22.28570532872236, 'p-value': 5.0849692257005624e-110}
Dimension 5: {'BDS statistic': 23.825065393472514, 'p-value': 1.8367221976840146e-125}
